In [ ]:
#!/usr/bin/python
# -*- coding: <encoding name> -*-
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/My Drive/gender_bias'

/content/drive/My Drive/gender_bias


In [ ]:
!pip install hanlp_restful     # 共指消解模型
!pip install metric4coref     # 评估方法

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 863 kB/s 
  Created wheel for hanlp-restful: filename=hanlp_restful-0.0.17-py3-none-any.whl size=8264 sha256=b3bc07ee31d851d1e406ac6c7e0275eb197f2516d89541ace49bc2c53d9f4621
  Stored in directory: /root/.cache/pip/wheels/45/08/f6/b45b93faff08691dff178e15befe39f477c9231bee46c1bfb2
  Created wheel for hanlp-common: filename=hanlp_common-0.0.15-py3-none-any.whl size=29762 sha256=39dc7c83d1446c3aad3b2adfabd20d47e7f00e335b2d3d4df45dec2318679606
  Stored in directory: /root/.cache/pip/wheels/90/3f/31/a488bc50d3e49a7c5b57ccf44e32c2fd66578ca52a731529ce
  Created wheel for phrasetree: filename=phrasetree-0.0.8-py3-none-any.whl size=44235 sha256=4ac22c6aa64bca1570432236e33a64145af3d765721bc10512bafbbd20614c7e
  Stored in directory: /root/.cache/pip/wheels/5c/00/7b/195f300c36302f4eff31f80f52c4b976247550a2af888b3f0c
Successfully built hanlp-restful hanlp-

In [ ]:
from hanlp_restful import HanLPClient
HanLP = HanLPClient('https://www.hanlp.com/api', auth='MTE4OEBiYnMuaGFubHAuY29tOjB5MXR5TXFRMlRIT3ZSM2c=', language='zh') # auth不填则匿名，zh中文，mul多语种

In [ ]:
import jieba
import time
import numpy as np
import re
from metric4coref import muc, ceaf, b_cubed, conll_coref_f1
from scipy import spatial

## 数据读取

In [ ]:
pro_dev=[]
with open('pro_stereotyped_type1_dev_ch.txt','r')as f:
    lines=f.readlines()
    for i,line in enumerate(lines):
        if i<99:
            pro_dev.append(line[2:-2])
        else:
            pro_dev.append(line[3:-2])

In [ ]:
re_pro_dev=[]
with open('re_wri_pro_stereotyped_type1_dev_ch.txt','r')as f:
    lines=f.readlines()
    for i,line in enumerate(lines):
        if i<99:
            re_pro_dev.append(line[2:-2])
        else:
            re_pro_dev.append(line[3:-2])

In [ ]:
anti_dev=[]
with open('anti_stereotyped_type1_dev_ch.txt','r',encoding='gbk')as f:
    lines=f.readlines()
    for i,line in enumerate(lines):
        if i<99:
            anti_dev.append(line[2:-2])
        else:
            anti_dev.append(line[3:-2])

## 提取数据的标注簇

In [ ]:
ori_anti_dev_c=[]
for dev in anti_dev:
  res = re.findall(r'\[(.*?)\]',dev)
  ori_anti_dev_c.append(res)

ori_pro_dev_c=[]
for dev in pro_dev:
  res = re.findall(r'\[(.*?)\]',dev)
  ori_pro_dev_c.append(res)

ori_re_pro_dev_c=[]
for dev in re_pro_dev:
  res = re.findall(r'\[(.*?)\]',dev)
  ori_re_pro_dev_c.append(res)
ori_anti_dev_c

In [ ]:
# 去除句子中的[]
anti_dev_clean=[]
for dev in anti_dev:
  dev=dev.replace('[','')
  dev=dev.replace(']','')
  dev=dev.replace(' ','')
  anti_dev_clean.append(dev)

In [ ]:
re_pro_dev_clean=[]
for dev in re_pro_dev:
  dev=dev.replace('[','')
  dev=dev.replace(']','')
  dev=dev.replace(' ','')
  re_pro_dev_clean.append(dev)

In [ ]:
pro_dev_clean=[]
for dev in pro_dev:
  dev=dev.replace('[','')
  dev=dev.replace(']','')
  dev=dev.replace(' ','')
  pro_dev_clean.append(dev)

In [ ]:
# re_pro共指消解
re_pro_res=[]
for i,dev in enumerate(re_pro_dev_clean):
  print(i)
  try:
    r = HanLP.coreference_resolution(dev)
  except:
    print('休息一下')
    time.sleep(40)
  re_pro_res.append(r)

In [ ]:
# anti共指消解
res=[]
for i,dev in enumerate(anti_dev_clean):
  print(i)
  try:
    r = HanLP.coreference_resolution(dev)
  except:
    print('休息一下')
    time.sleep(40)
  res.append(r)

In [ ]:
# pro共指消解
pro_res=[]
for i,dev in enumerate(pro_dev_clean):
  print(i)
  try:
    r = HanLP.coreference_resolution(dev)
  except:
    print('休息一下')
    time.sleep(40)
  pro_res.append(r)

In [ ]:
# np.save('anti_res.npy', res) # 注意带上后缀名
# np.save('pro_res.npy', pro_res) # 注意带上后缀名
# np.save('re_pro_res.npy', re_pro_res) # 注意带上后缀名

## 共指消解结果评估

In [ ]:
anti_res=np.load('anti_res.npy',allow_pickle=True)
pro_res=np.load('pro_res.npy',allow_pickle=True)
re_pro_res=np.load('re_pro_res.npy',allow_pickle=True)
anti_res

In [ ]:
re_pro_cluster=[]
for r in re_pro_res:
  re_pro_cluster.append(r['clusters'])

pro_cluster=[]
for r in pro_res:
  pro_cluster.append(r['clusters'])

In [ ]:
cluster=[]
for r in anti_res:
  cluster.append(r['clusters'])
cluster

In [ ]:
eval_anti_c=[]
eval_anti_0=[]
for c in cluster:
  try:
    eval_anti_c.append([c[0][0][0],c[0][1][0]])
    eval_anti_0.append(c[0][0][0])
  except:
    eval_anti_c.append([])
    eval_anti_0.append('')

eval_pro_c=[]
eval_pro_0=[]
for c in pro_cluster:
  try:
    eval_pro_c.append([c[0][0][0],c[0][1][0]])
    eval_pro_0.append(c[0][0][0])
  except:
    eval_pro_c.append([])
    eval_pro_0.append('')

eval_re_pro_c=[]
eval_re_pro_0=[]
for c in re_pro_cluster:
  try:
    eval_re_pro_c.append([c[0][0][0],c[0][1][0]])
    eval_re_pro_0.append(c[0][0][0])
  except:
    eval_re_pro_c.append([])
    eval_re_pro_0.append('')

In [ ]:
# 计算欧氏距离，证明模型无偏
# 去偏前
count_0=0
for x,y in zip(eval_anti_0,eval_pro_0):
  if x!=y:
    count_0+=1
edistance_0=count_0/len(eval_anti_0)
print(edistance_0)

# 去偏后
count_1=0
for x,y in zip(eval_anti_0,eval_re_pro_0):
  if x!=y:
    count_1+=1
edistance_1=count_1/len(eval_anti_0)
print(edistance_1)

0.3383838383838384
0.30808080808080807


In [ ]:
# 证明性能保持
print(b_cubed(ori_pro_dev_c, eval_anti_c))     # 去偏前的准确性
print(b_cubed(ori_pro_dev_c, eval_re_pro_c))     # 去偏后的准确性

(0.776595744680851, 0.776595744680851, 0.776595744680851)
(0.79, 0.79, 0.79)
